## Report

## 1. 先將資料透過readData轉成(out_node, in_node)的形式紀錄edge，然後分別紀錄inlink跟outlink的node，透過union得到總共的node

In [10]:
import pyspark


conf = pyspark.SparkConf().setMaster('local').setAppName('pagerank')
sc = pyspark.SparkContext(conf=conf)

distFile = sc.textFile("input.txt")


def readData(x):
    l = x.split('\t')
    res = []
    res.append((l[0], l[1]))
    return res


def buildMartix(x):
    col = []
    for i in x[1]:
        length = len(x[1])
        col.append((x[0], (i, 1/length)))

    return (col)


rdd = distFile.flatMap(readData)

outlinks = rdd.keys().distinct()
inlinks = rdd.map(lambda x: x[1]).distinct()

nodes = sc.union([outlinks, inlinks]).keyBy(lambda x: x).sortByKey().distinct()
nodes = nodes.map(lambda x: x[1])
nodes_num = nodes.count()


## 2. 將outlink的node跟inlink的node做substract可以得到只有outlink的node，並新增一個連到自己的edge，weight為0
##    把相同outlink的node用groupByKey合在一起，資料變成這種形式(out, [in1, in2, in3....])

In [11]:
only_out_nodes = outlinks.subtract(inlinks)
only_out_nodes = only_out_nodes.map(lambda x: (x, (x, 0)))
rdd = rdd.groupByKey().mapValues(list)

## 3. 接著可以用連出去的edge的總數算出每條edge該有的weight
## 再將原始的資料跟只有outlink的node union在一起，就可以得到原始的matrix
## 將matrix乘上beta值後，把資料轉成(out_node, (in_node, weight))

In [12]:
beta = 0.8
r = nodes.map(lambda x: (x, (1/nodes_num)))
matrix = rdd.flatMap(buildMartix)
matrix = sc.union([matrix, only_out_nodes]).sortByKey()
matrix = matrix.map(lambda x: (x[0], (x[1][0], x[1][1] * beta)))

_A = matrix.map(lambda x: (x[0], (x[1][0], x[1][1])))

## 4. 最後是進行iteration，在算出每一次iterate後的r，因為會有dead-end的可能，所以要更新r的值
## 把最終的r用value做排列，最後輸出前20個node

In [13]:

def productSum(x):
    res = 0
    for i in range(len(x[0][1])):
        res += (x[0][1][i] * x[1][i])
    return res


for i in range(20):
    r = _A.join(r).map(lambda x: (
        x[1][0][0], x[1][0][1] * x[1][1])).reduceByKey(lambda x, y: x+y)
    _sum = r.values().sum()
    r = r.map(lambda x: (x[0], x[1]+((1 - _sum) / nodes_num)))

r = r.sortBy(lambda x: x[1], ascending=False)

res = r.take(10)
with open('Outputfile.txt', 'w+') as f:
    for i in res:
        f.write(f'{i[0]}\t{i[1]:.6f}\n')
        print(f'{i[0]}\t{i[1]:.6f}\n')

sc.stop()

4240	0.000632

10861	0.000629

6899	0.000524

9526	0.000512

2118	0.000496

3419	0.000485

1311	0.000480

3186	0.000470

3541	0.000463

367	0.000462

